# Tomography

First, some imports.

In [ ]:
import collections
import csv
import itertools
import pathlib
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy import optimize, sparse

These are paths to the data files. We need two things:

* Probes: This is a CSV file with headers `id` (string), `latitude` (float), and `longitude` (float).
* Links: This is a CSV file with headers `source_id` (string), `target_id` (string), `throughput` (float). The two id fields need to be found in the probes file.

In [ ]:
filepath_probes = pathlib.PurePath('probes.csv')
filepath_links = pathlib.PurePath('links.csv')

With these data files, we can generate a NetworkX graph. First, though, we need a few utility functions for dealing with geographic data.

In [ ]:
def mercator(longitude = None, latitude = None):
    """
    Given a longitude in [-180, 180] and a latitude in [-90, 90], return
    an (x, y) pair representing the location on a Mercator projection.
    Assuming the latitude is no larger/smaller than +/- 85
    (approximately), the pair will lie in [-0.5, 0.5]^2.
    """
    x = longitude / 360. if longitude is not None else None
    y = np.log(np.tan(np.pi / 4. + latitude * np.pi / 360.)) / (2. * np.pi) if latitude is not None else None

    x = float(x)
    y = float(y)

    if x is None:
        return y
    if y is None:
        return x
    return (x, y)

def get_sphere_point(latlong):
    """Convert spherical coordinates to Cartesian coordinates."""
    latlong = np.array(latlong) * np.pi / 180.
    return np.array([
        np.cos(latlong[1]) * np.cos(latlong[0]),
        np.sin(latlong[1]) * np.cos(latlong[0]),
        np.sin(latlong[0])
    ])

def get_spherical_distance(a, b):
    """Find the distance on the unit sphere between two unit vectors."""
    # The value of (a @ b) is clamped between -1 and 1 to avoid issues
    # with floating point
    if np.all(a == b):
        return np.float64(0.)
    return np.arccos(min(max(a @ b, -1.), 1.))

def get_GCL(latlong_a, latlong_b):
    """Find the great circle latency between two points on Earth in ms."""
    # The following are in meters and meters per second
    circumference_earth = 40075016.68557849
    radius_earth = circumference_earth / (2. * np.pi)
    c = 299792458.

    # Convert spherical coordinates to Cartesian coordinates
    p_a = get_sphere_point(latlong_a)
    p_b = get_sphere_point(latlong_b)

    # Special case for slightly improved numerical stability
    if np.all(p_a == p_b):
        return 0.

    # Compute the latency, which is the travel time at the rate of two
    # thirds the speed of light
    return 2. * 1000. * get_spherical_distance(p_a, p_b) * radius_earth \
        / (2. * c / 3.)

We also need a function to deal with co-located nodes. This is not important for our tomography computations, but it will matter for the Ricci curvature later.

In [ ]:
def cluster_graph(graph: nx.DiGraph):
    # Cluster by co-location
    clusters = collections.defaultdict(list)
    for node, data_node in graph.nodes(data=True):
        latitude = data_node['latitude']
        longitude = data_node['longitude']
        clusters[(latitude, longitude)].append(node)

    # Make mappings from cluster representatives to constituents and
    # vice versa
    representative_to_constituents = {
        min(cluster): cluster
        for cluster in clusters.values()
    }
    constituent_to_representative = {
        constituent: representative
        for representative, constituents in representative_to_constituents.items()
        for constituent in constituents
    }

    graph_new = nx.DiGraph()
    for representative in representative_to_constituents.keys():
        graph_new.add_node(
            representative,
            latitude=graph.nodes[representative]['latitude'],
            longitude=graph.nodes[representative]['longitude']
        )

    for u, v, data_u_v in graph.edges(data=True):
        representative_u = constituent_to_representative[u]
        representative_v = constituent_to_representative[v]

        if representative_u == representative_v:
            continue

        if (representative_u, representative_v) in graph_new.edges:
            # Aggregate with existing data
            graph_new.edges[representative_u, representative_v]['latency'] \
                = min(
                    graph_new.edges[representative_u, representative_v]['latency'],
                    data_u_v['latency']
                )
            graph_new.edges[representative_u, representative_v]['throughput'] \
                = graph_new.edges[representative_u, representative_v]['throughput'] \
                    + data_u_v['latency']
        else:
            graph_new.add_edge(
                representative_u, representative_v,
                latency=data_u_v['latency'],
                throughput=data_u_v['throughput']
            )

    return graph_new

Now we make the graph, as well as save some supplementary information. Importantly, we need an association between graph edges and indices. We also bookkeep the amount of data flowing into and out of each node.

In [ ]:
graph = nx.DiGraph()
with open(filepath_probes, 'r') as file_probes:
    reader = csv.DictReader(file_probes)
    for index, row in enumerate(reader):
        graph.add_node(
            row['id'],
            latitude=float(row['latitude']),
            longitude=float(row['longitude'])
        )

with open(filepath_links, 'r') as file_links:
    reader = csv.DictReader(file_links)
    for row in reader:
        id_source = row['source_id']
        id_target = row['target_id']

        node_source = graph.nodes[id_source]
        node_target = graph.nodes[id_target]

        graph.add_edge(
            row['source_id'], row['target_id'],
            throughput=float(row['throughput']),
            latency=get_GCL(
                (node_source['latitude'], node_source['longitude']),
                (node_target['latitude'], node_target['longitude']),
            )
        )

graph = cluster_graph(graph)

index_to_link_id = []
link_id_to_index = {}
traffic_out_per_node = collections.defaultdict(float)
traffic_in_per_node = collections.defaultdict(float)
traffic_total = 0.
for index, (id_source, id_target, throughput) in enumerate(graph.edges.data('throughput')):
    index_to_link_id.append((id_source, id_target))
    link_id_to_index[(id_source, id_target)] = index

    traffic_out_per_node[id_source] += throughput
    traffic_in_per_node[id_target] += throughput
    traffic_total += throughput

Now we start solving a tomography problem. Our strategy is to follow the ideas from [An information-theoretic approach to traffic matrix estimation](https://dl.acm.org/doi/10.1145/863955.863990).

Our first goal is to determine how much traffic is going between each source-destination pair in our network. For simplicity, assume that each of these pairs define a single route (say, by shortest-path routing). This should be a pretty easy assumption to adapt around in the future if we want, but it makes this demonstration a bit easier.

Let $A$ be our routing matrix. $A$ is $n \times p$, where $n$ is the size of the observed data (link utilizations), and $p$ is the number of possible routes (number of source-destination pairs*). We also define $y$, an $n$-vector containing the link utilizations**. We set $A_{i, j}$ to $1$ if the $i$ th link is used in the $j$ th route, and $0$ otherwise. Note that $A$ is sparse.

Our goal is to solve the (underconstrained) linear problem $y = Ax$ for the $p$-vector $x$.

*We actually don't need every possible pair. There are some obvious pairs that will have no associated traffic. For example, if a node has no outgoing (incoming) traffic, then any route with it as a source (destination) will have no traffic. So we can remove the corresponding columns of $A$ and elements of $x$.

**Here and elsewhere, we actually scale $y$ and $x$ by the total traffic in the system. It turns out that this makes it easier on the optimization algorithms.

In [ ]:
# This is a scaling of y by the total traffic in the system
traffic_counts = np.array([
    graph.edges[source_id, target_id]['throughput']
    for source_id, target_id in index_to_link_id
])

# Determine the ordering of the columns of A, ignoring
# source-destination pairs with no possible traffic
sources, destinations = zip(*[
    (source, destination)
    for source in graph.nodes
    for destination in graph.nodes
    if (
        source != destination
        and traffic_out_per_node[source] > 0.
        and traffic_in_per_node[destination] > 0.
    )
])

# Also have a mapping to go from source-destination pairs to indices
source_destination_to_index = {
    (source, destination): index
    for index, (source, destination) in enumerate(zip(sources, destinations))
}

# Compute routes between each source-destination pair. Assume shortest
# path routing for this example.
routes = {
    source: nx.single_source_dijkstra_path(graph, source, weight='latency')
    for source in graph.nodes
}

# Create the (sparse) traffic matrix and its transpose
traffic_matrix_data = []
traffic_matrix_row_ind = []
traffic_matrix_col_ind = []
for index, (source, destination) in enumerate(zip(sources, destinations)):
    route = routes[source][destination]
    for link_id in itertools.pairwise(route):
        traffic_matrix_data.append(1)
        traffic_matrix_row_ind.append(link_id_to_index[link_id])
        traffic_matrix_col_ind.append(index)

traffic_matrix = sparse.csr_matrix(
    (traffic_matrix_data, (traffic_matrix_row_ind, traffic_matrix_col_ind)),
    shape=(len(index_to_link_id), len(sources))
)
traffic_matrix_transpose = sparse.csr_matrix(
    (traffic_matrix_data, (traffic_matrix_col_ind, traffic_matrix_row_ind)),
    shape=(len(sources), len(index_to_link_id))
)

Ideally, we want to find $x$ such that $y = Ax$ that minimizes some objective function $J(x)$. We do this by solving the "soft" problem of minimizing $f(x) = \|y - Ax\|_2^2 + \lambda J(x)$ across choices of $x$, where $\lambda$ is a tunable hyperparameter.

For our choice of $J$, we use the mutual information between $S$ and $D$, where $S$ is a random variable representing a unit of traffic's source, and $D$ represents its destination.

We define $\mathcal{L}(x)$ and its gradient $\nabla \mathcal{L}(x)$ below. It turns out that a completely accurate implementation is somewhat complex (provided with `loss_alt` and `dif_loss_alt`). We instead use the approximations given by `loss` and `dif_loss`. Some experimentation reveals that the outputs are comparable, but `loss` allows for a significantly faster output.

In [ ]:
def loss(xs, lam=0.01):
    errors = traffic_counts - traffic_total * (traffic_matrix @ xs)
    accuracy = errors @ errors

    penalty = 0.
    if lam != 0.:
        for x, source, destination in zip(xs, sources, destinations):
            n_s = traffic_out_per_node[source]
            n_d = traffic_in_per_node[destination]
            if n_s > 0. and n_d > 0. and x != 0.:
                penalty += x * np.log2(x * traffic_total**2 / (n_s * n_d))

    if np.isinf(lam):
        return penalty

    return accuracy / traffic_total**2 + lam**2 * penalty

def dif_loss(xs, lam=0.01):
    errors = traffic_counts - traffic_total * (traffic_matrix @ xs)
    dif_accuracy = -2 * traffic_total * traffic_matrix_transpose @ errors

    if lam != 0.:
        dif_penalty = np.array([
            np.log2(x * traffic_total**2 / (n_s * n_d)) + 1 / np.log(2)
            if n_s > 0. and n_d > 0. else 0.
            for x, source, destination in zip(xs, sources, destinations)
            for n_s in (traffic_out_per_node[source],)
            for n_d in (traffic_in_per_node[destination],)
        ])
    else:
        dif_penalty = np.zeros(xs.shape)

    return dif_accuracy / traffic_total**2 + lam**2 * dif_penalty

In [ ]:
def loss_alt(xs, lam=0.01):
    p_s = collections.defaultdict(float)
    p_d = collections.defaultdict(float)
    x_sum = 0.
    for x, source, destination in zip(xs, sources, destinations):
        p_s[source] += x
        p_d[destination] += x
        x_sum += x

    errors = (traffic_counts / traffic_total) - traffic_matrix @ xs
    accuracy = errors @ errors

    penalty = 0.
    if lam != 0.:
        for x, source, destination in zip(xs, sources, destinations):
            p_s_source = p_s[source]
            p_d_destination = p_d[destination]
            if p_s_source > 0. and p_d_destination > 0. and x != 0.:
                penalty += (x / x_sum) * np.log2(x * x_sum / (p_s_source * p_d_destination))

    if np.isinf(lam):
        return penalty

    return accuracy + lam**2 * penalty

def dif_loss_alt(xs, lam=0.01):
    p_s = collections.defaultdict(float)
    p_d = collections.defaultdict(float)
    x_sum = 0.
    source_to_indices = collections.defaultdict(set)
    destination_to_indices = collections.defaultdict(set)
    for index, (x, source, destination) in enumerate(zip(xs, sources, destinations)):
        p_s[source] += x
        p_d[destination] += x
        x_sum += x
        source_to_indices[source].add(index)
        destination_to_indices[destination].add(index)

    errors = (traffic_counts / traffic_total) - traffic_matrix @ xs
    dif_accuracy = -2 * traffic_matrix_transpose @ errors

    dif_penalty = np.zeros(xs.shape)
    if lam != 0.:
        for index, (x, source, destination) in enumerate(zip(xs, sources, destinations)):
            p_s_source = p_s[source]
            p_d_destination = p_d[destination]
            if p_s_source > 0. and p_d_destination > 0. and x != 0.:
                for index_dif in range(len(xs)):
                    if index_dif in source_to_indices[source]:
                        if index_dif in destination_to_indices[destination]:
                            if index == index_dif:
                                dif_penalty[index_dif] += ((x_sum - x) / x_sum**2) * np.log2(x * x_sum / (p_s_source * p_d_destination)) \
                                    + ((x_sum + x) * p_s_source * p_d_destination - x * x_sum * (p_s_source + p_d_destination)) / (np.log(2) * x_sum**2 * p_s_source * p_d_destination)
                            else:
                                pass  # Assume no duplicate source-destination pairs
                        else:
                            dif_penalty[index_dif] += -(x / x_sum**2) * np.log2(x * x_sum / (p_s_source * p_d_destination)) \
                                + x * (p_s_source - x_sum) / (np.log(2) * x_sum**2 * p_s_source)
                    else:
                        if index_dif in destination_to_indices[destination]:
                            dif_penalty[index_dif] += -(x / x_sum**2) * np.log2(x * x_sum / (p_s_source * p_d_destination)) \
                                + x * (p_d_destination - x_sum) / (np.log(2) * x_sum**2 * p_d_destination)
                        else:
                            dif_penalty[index_dif] += -(x / x_sum**2) * np.log2(x * x_sum / (p_s_source * p_d_destination)) \
                                + x / (np.log(2) * x_sum**2)

    if np.isinf(lam):
        return dif_penalty

    return dif_accuracy + lam**2 * dif_penalty

We initialize the optimization using a simple gravity model. From there, we use a L-BFGS-B minimizer to minimize the objective function. (The source paper uses a convex optimizer in MATLAB, but SciPy's convex optimizer doesn't behave well with this objective function for some reason.) Note that the prevalence of logs in the regularization term requires setting the lower bounds on the variables to a small but positive number.

In [ ]:
# Gravity model
x_0 = np.array([
    n_s * n_d / traffic_total**2
    for source, destination in zip(sources, destinations)
    for n_s in (traffic_out_per_node[source],)
    for n_d in (traffic_in_per_node[destination],)
])
x_0 = x_0 / sum(x_0)

x, _, _ = optimize.fmin_l_bfgs_b(
    loss, x_0, fprime=dif_loss, args=[0.01],
    # factr=1e1, pgtol=1e-12,  # Tuning parameters for the optimizer
    bounds=[(1e-12, 1.) for _ in x_0]
)

Here is a histogram of the logs of the traffic estimates between each source-destination pair. Note that very few elements are set to the lower bound.

In [ ]:
plt.hist(np.log(x * traffic_total), bins=100)
plt.xlabel('Log of traffic')
plt.ylabel('Count')
plt.show()

And here is a verification that the value we computed actually reflects the measured link utilizations. A log-log plot is used for clarity.

In [ ]:
fig, ax = plt.subplots(1, 1)
plot_x = [np.log(traffic_count + 1) for traffic_count in traffic_counts]
plot_y = [np.log(traffic_estimated + 1) for traffic_estimated in (traffic_total * (traffic_matrix @ x))]
ax.plot(plot_x, plot_y, 'b.')
ax.set_xlabel('Actual')
ax.set_ylabel('From Estimated Flows')
ax.set_aspect('equal')
plt.show()

# Curvature

Now that we have (approximate) flow-level measurements between nodes, we compute the flow-associated distances between the neighborhoods of nodes (i.e., the earth mover's distance used in the computation of Ollivier-Ricci curvature).

The original idea is to place a certain total amount of mass $1 - \alpha$ on the neighbors of a node $u$ and transport it to the neighbors of an adjacent node $v$. For simplicity, let's use $\alpha = 0$. In the future, we can make a simple adaptation to allow for other $\alpha$ values, but it is not immediately clear how the behavior will be as we take $\alpha \rightarrow 1$.

There are two pieces to fill in, both potentially helped by the computed flows:


* What do the distributions of mass look like? When considering Ricci curvature of the edge $u \rightarrow v$, we only care about the bytes that flow from neighbors of $u$ to neighbors of $v$.

  Define $f^{u \rightarrow v}_s$ to be the amount of flow from a neighbor $s \ne v$ of $u$ that eventually makes its way to a neighbor of $v$. In other words, we are summing flows associated to paths of the form $\cdots \rightarrow s \rightarrow \cdots \rightarrow t \rightarrow \cdots$ where $t \ne u$ is a neighbor of $v$.

  Similarly, define $g^{u \rightarrow v}_t$ to be the amount of flow passing through a neighbor of $u$ that eventually makes its way to $t \ne u$. Here we are summing flows associated to paths of the form $\cdots \rightarrow s \rightarrow \cdots \rightarrow t \rightarrow \cdots$ where $s \ne v$ is a neighbor of $u$.

  Note that we are explicitly enforcing $s \ne v$ and $t \ne u$ to avoid the same "shortcutting" problem the original Ollivier-Ricci curvature suffers from when defined on graphs.

  By construction, we have
  $$\sum_{\substack{s \\ s \rightarrow u \\ s \ne v}} f^{u \rightarrow v}_s = \sum_{\substack{t \\ v \rightarrow t \\ t \ne u}} g^{u \rightarrow v}_t,$$
  as both quantities are equivalent to
  $$M := \sum_{\substack{s \\ s \to u \\ s \ne v}} \sum_{\substack{t \\ v \to t \\ t \ne u}} \sum_{\substack{p \\ (s \twoheadrightarrow t) \in p}} x_p,$$
  where $(s \twoheadrightarrow t) \in p$ signifies that $p$ is a path of the form $\cdots \rightarrow s \rightarrow \cdots \rightarrow t \rightarrow \cdots$, and $x_p$ is amount of flow associated to $p$.

  We now are prepared to define our two distributions. The source distribution simply places weights $\frac{1}{M} f^{u \rightarrow v}_s$ on each $s \rightarrow u$ with $s \ne v$; the target distribution puts mass $\frac{1}{M} f^{u \rightarrow v}_t$ on each $t$ with $v \rightarrow v$ with $t \ne u$. Our fact about $M$ shows that these distributions are truly allocations of a single unit of mass.

* What does the transportation plan look like? The intuition here is that we want the solution to the transport problem to look like the actual flows we measured. The construction given in the previous bullet point allows exactly this. For a neighbor $s \ne v$ of $u$, each unit of mass is associated to a path $\cdots \rightarrow s \rightarrow \cdots \rightarrow t \rightarrow \cdots$, and we can therefore flow that mass from $s$ to $t$ along the (sub-)path. By construction, this plan does actually map the source distribution to the target distribution.

The cost of this transportation plan is
$$c(u, v) = \frac{1}{M} \sum_{\substack{s \\ s \to u \\ s \ne v}} \sum_{\substack{t \\ v \to t \\ t \ne u}} \sum_{\substack{p \\ (s \twoheadrightarrow t) \in p}} x_p d_p(s, t),$$
where $d_p(s, t)$ is the distance from $s$ to $t$ when following $p$. This is readily computed with the following ideas:

* Iteration over the paths $p$ is done using the same routing data used to construct the traffic matrix. (This is just shortest paths in our case.)

* The $x_p$ values are actually just elements of the vector $x$ normalized so that they sum to $1$, where $x$ is the vector found by the earlier tomography step. Some cancellation of fractions reveals that we can just treat $x_p$'s as the elements of $x$.

* $d_p(s, t)$ can be computed by following $p$ and either counting the number of hops from $s$ to $t$ or summing measured or approximate (i.e., great circle) latencies.

In [ ]:
# Both of these are mappings from s-t pairs to their associated values
# (denominator and numerator)
x_sum = collections.defaultdict(float)
xd_sum = collections.defaultdict(float)

for source, routes_source in routes.items():
    for destination, route in routes_source.items():
        if (source, destination) not in source_destination_to_index:
            # In this case, x_p = 0
            continue

        x_p = x[source_destination_to_index[(source, destination)]]
        for index_s, s in enumerate(route[:-1]):
            d_p_s_t = 0.
            for t_previous, t in itertools.pairwise(route[index_s:]):
                d_p_s_t += graph.edges[t_previous, t]['latency']

                x_sum[(s, t)] += x_p
                xd_sum[(s, t)] += x_p * d_p_s_t

transportation_costs = {}
for u, v in graph.edges:
    denominator = 0.
    numerator = 0.
    for s in graph.predecessors(u):
        if s == v:
            continue
        for t in graph.successors(v):
            if t == u:
                continue

            denominator += x_sum[(s, t)]
            numerator += xd_sum[(s, t)]
    transportation_costs[(u, v)] = numerator / denominator if denominator != 0. else 3. * graph.edges[u, v]['latency']
    if denominator == 0.:
        print(f'Manually setting curvature of {u} -> {v} to {-2.}')

Now we can compute the Ricci curvatures
$$\kappa_{u \rightarrow v} = 1 - \frac{c(u, v)}{d(u, v)}.$$

In [ ]:
for u, v in graph.edges:
    graph.edges[u, v]['curvature'] = 1. - transportation_costs[(u, v)] / graph.edges[u, v]['latency']

There are many notable aspects of these curvatures. First, note that the curvature is defined on a directed graph. This turns out to be fine because the curvatures are rather symmetric across opposite edges. Note that almost all of the points in the following plot lie very close to the line $x = y$:

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(
    [graph.edges[u, v]['curvature'] for (u, v) in graph.edges if (v, u) in graph.edges],
    [graph.edges[v, u]['curvature'] for (u, v) in graph.edges if (v, u) in graph.edges],
    'b.'
)
ax.set_aspect('equal')
ax.set_xlabel(r'Curvature on $u \rightarrow v$')
ax.set_ylabel(r'Curvature on $v \rightarrow u$')
plt.show()

Also, it is rather clear that a large proportion of the curvatures are negative. This aligns with our intuition, as the given network graph consists of critical backbone links. However, it might be challenging for our manifold optimization algorithm to work with.

In [ ]:
plt.hist([curvature for _, _, curvature in graph.edges.data('curvature')], bins=20)
plt.xlabel('Curvature')
plt.ylabel('Count')
plt.show()

Finally, there doesn't seem to be much direct correlation between throughput and curvature:

In [ ]:
plt.plot(
    [np.log(1. + graph.edges[u, v]['throughput']) for (u, v) in graph.edges],
    [graph.edges[u, v]['curvature'] for (u, v) in graph.edges],
    'b.'
)
plt.xlabel('Log of Throughput')
plt.ylabel('Curvature')
plt.show()

In [ ]:
def get_network_plot(
    graph,
    ax = None,
    weight_label='curvature', color_min=-2., color_max=2., colormap='RdBu'
):
    if ax is None:
        fig, ax = plt.subplots(1, 1, facecolor='#808080')
    else:
        fig = ax.get_figure()
    ax.set_aspect('equal')
    ax.axis('off')

    # Plot the edges
    for u, v, d in graph.edges(data=True):
        if (v, u) in graph.edges:
            # Don't double-draw edges
            if u > v:
                continue

            # For edges where an opposite exists, just draw the average
            # value
            weight = (d[weight_label] + graph.edges[v, u][weight_label]) / 2.
        else:
            weight = d[weight_label]
        color = mpl.colormaps[colormap]((weight - color_min) / (color_max - color_min))

        x_u, y_u = mercator(graph.nodes[u]['longitude'], graph.nodes[u]['latitude'])
        x_v, y_v = mercator(graph.nodes[v]['longitude'], graph.nodes[v]['latitude'])
        ax.plot([x_u, x_v], [y_u, y_v], color=color)

    # Plot the vertices
    for node, d in graph.nodes(data=True):
        # If trim_vertices is set, then only plot the vertices with
        # incident edges
        if graph[node]:
            x, y = mercator(graph.nodes[node]['longitude'],
                            graph.nodes[node]['latitude'])
            ax.plot(x, y, '.', ms=4, color='green')

    return fig

for u, v, throughput in graph.edges.data('throughput'):
    graph.edges[u, v]['log_throughput'] = np.log(throughput + 1.)

fig, (ax_1, ax_2) = plt.subplots(2, 1, facecolor='#808080')
get_network_plot(
    graph, ax_1, 'log_throughput',
    0., max([log_throughput for _, _, log_throughput in graph.edges.data('log_throughput')]),
    colormap='PiYG')
get_network_plot(graph, ax_2)
ax_1.set_title('Log Throughputs (Green = High)')
ax_2.set_title('Curvatures (Red = Negative)')
plt.show()